# RUN WAY

In [1]:
from tools.infer.predict_system import run

In [2]:
#Lite models
run(image_dir = './doc/imgs/00018069.jpg' , 
    det_model_dir='C://Users//Jim//.paddleocr//whl//det//ch//ch_PP-OCRv4_det_infer',
    rec_model_dir = "C://Users//Jim//.paddleocr//whl//rec//ch//ch_PP-OCRv4_rec_infer",
    use_angle_cls = False)

[2023/12/20 18:29:27] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2023/12/20 18:29:27] ppocr DEBUG: dt_boxes num : 69, elapsed : 0.18650245666503906
[2023/12/20 18:29:36] ppocr DEBUG: rec_res num  : 69, elapsed : 8.864979267120361
[2023/12/20 18:29:36] ppocr DEBUG: 0  Predict time of ./doc/imgs/00018069.jpg: 9.052s
[2023/12/20 18:29:36] ppocr DEBUG: 代号, 1.000
[2023/12/20 18:29:36] ppocr DEBUG: 项目, 1.000
[2023/12/20 18:29:36] ppocr DEBUG: 结果, 0.999
[2023/12/20 18:29:36] ppocr DEBUG: 参考值, 1.000
[2023/12/20 18:29:36] ppocr DEBUG: 单位, 1.000
[2023/12/20 18:29:36] ppocr DEBUG: ALT, 0.997
[2023/12/20 18:29:36] ppocr DEBUG: 谷丙转氨酶, 0.998
[2023/12/20 18:29:36] ppocr DEBUG: 25.6, 0.998
[2023/12/20 18:29:36] ppocr DEBUG: 0--40, 0.960
[2023/12/20 18:29:36] ppocr DEBUG: U/L, 0.988
[2023/12/20 18:29:36] ppocr DEBUG: TBIL, 0.995
[2023/12/20 18:29:36] ppocr DEBU

In [6]:
#Server models
run(image_dir = './doc/imgs/00018069.jpg' , 
    det_model_dir='C://AI//Models//paddleOCR//ch_PP-OCRv4_det_server_infer',
    rec_model_dir = "C://AI//Models//paddleOCR//ch_PP-OCRv4_rec_server_infer",
    use_angle_cls = False)


[2023/12/20 18:58:51] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2023/12/20 18:58:54] ppocr DEBUG: dt_boxes num : 73, elapsed : 3.1983482837677
[2023/12/20 18:59:50] ppocr DEBUG: rec_res num  : 73, elapsed : 55.83746528625488
[2023/12/20 18:59:50] ppocr DEBUG: 0  Predict time of ./doc/imgs/00018069.jpg: 59.043s
[2023/12/20 18:59:50] ppocr DEBUG: 代号, 0.998
[2023/12/20 18:59:50] ppocr DEBUG: 项目, 0.999
[2023/12/20 18:59:50] ppocr DEBUG: 结果, 0.999
[2023/12/20 18:59:50] ppocr DEBUG: 参考值, 0.997
[2023/12/20 18:59:50] ppocr DEBUG: 单位, 1.000
[2023/12/20 18:59:50] ppocr DEBUG: ALT, 0.995
[2023/12/20 18:59:50] ppocr DEBUG: 谷丙转氨酶, 0.998
[2023/12/20 18:59:50] ppocr DEBUG: 25.6, 0.996
[2023/12/20 18:59:50] ppocr DEBUG: 0--40, 0.957
[2023/12/20 18:59:50] ppocr DEBUG: U/L, 0.977
[2023/12/20 18:59:50] ppocr DEBUG: TBIL, 0.995
[2023/12/20 18:59:50] ppocr DEBUG: 

In [4]:
from types import SimpleNamespace
d = {
    'image_dir':'./doc/imgs/00018069.jpg',
    'det_model_dir':'C://Users//Jim//.paddleocr//whl//det//ch//ch_PP-OCRv4_det_infer',
    'rec_model_dir': 'C://Users//Jim//.paddleocr//whl//rec//ch//ch_PP-OCRv4_rec_infer',
    'use_angle_cls': 'False'
}
args = SimpleNamespace(**d)
print(args.image_dir)  # 输出 'value1'

./doc/imgs/00018069.jpg


In [5]:
type(args)

types.SimpleNamespace

# Serving

In [ ]:
# pip install paddle-serving-server
# pip install paddle-serving-client

In [2]:
from paddle_serving_client import Client
import cv2
import sys
import numpy as np
import os
import time
import re
import base64
from tools.infer.predict_cls import TextClassifier
from params import read_params

global_args = read_params()
from paddle_serving_server_gpu.web_service import WebService


ModuleNotFoundError: No module named 'params'

In [ ]:

class TextClassifierHelper(TextClassifier):
    def __init__(self, args):
        self.cls_image_shape = [int(v) for v in args.cls_image_shape.split(",")]
        self.cls_batch_num = args.rec_batch_num
        self.label_list = args.label_list
        self.cls_thresh = args.cls_thresh
        self.fetch = [
            "save_infer_model/scale_0.tmp_0", "save_infer_model/scale_1.tmp_0"
        ]

    def preprocess(self, img_list):
        args = {}
        img_num = len(img_list)
        args["img_list"] = img_list
        # Calculate the aspect ratio of all text bars
        width_list = []
        for img in img_list:
            width_list.append(img.shape[1] / float(img.shape[0]))
        # Sorting can speed up the cls process
        indices = np.argsort(np.array(width_list))
        args["indices"] = indices
        cls_res = [['', 0.0]] * img_num
        batch_num = self.cls_batch_num
        predict_time = 0
        beg_img_no, end_img_no = 0, img_num
        norm_img_batch = []
        max_wh_ratio = 0
        for ino in range(beg_img_no, end_img_no):
            h, w = img_list[indices[ino]].shape[0:2]
            wh_ratio = w * 1.0 / h
            max_wh_ratio = max(max_wh_ratio, wh_ratio)
        for ino in range(beg_img_no, end_img_no):
            norm_img = self.resize_norm_img(img_list[indices[ino]])
            norm_img = norm_img[np.newaxis, :]
            norm_img_batch.append(norm_img)
        norm_img_batch = np.concatenate(norm_img_batch)
        if img_num > 1:
            feed = [{
                "image": norm_img_batch[x]
            } for x in range(norm_img_batch.shape[0])]
        else:
            feed = {"image": norm_img_batch[0]}
        return feed, self.fetch, args

    def postprocess(self, outputs, args):
        prob_out = outputs[0]
        label_out = outputs[1]
        indices = args["indices"]
        img_list = args["img_list"]
        cls_res = [['', 0.0]] * len(label_out)
        if len(label_out.shape) != 1:
            prob_out, label_out = label_out, prob_out
        for rno in range(len(label_out)):
            label_idx = label_out[rno]
            score = prob_out[rno][label_idx]
            label = self.label_list[label_idx]
            cls_res[indices[rno]] = [label, score]
            if '180' in label and score > self.cls_thresh:
                img_list[indices[rno]] = cv2.rotate(img_list[indices[rno]], 1)
        return img_list, cls_res



In [ ]:

class OCRService(WebService):
    def init_rec(self):
        self.text_classifier = TextClassifierHelper(global_args)

    def preprocess(self, feed=[], fetch=[]):
        # TODO: to handle batch rec images
        img_list = []
        for feed_data in feed:
            data = base64.b64decode(feed_data["image"].encode('utf8'))
            data = np.fromstring(data, np.uint8)
            im = cv2.imdecode(data, cv2.IMREAD_COLOR)
            img_list.append(im)
        feed, fetch, self.tmp_args = self.text_classifier.preprocess(img_list)
        return feed, fetch

    def postprocess(self, feed={}, fetch=[], fetch_map=None):
        outputs = [fetch_map[x] for x in self.text_classifier.fetch]
        for x in fetch_map.keys():
            if ".lod" in x:
                self.tmp_args[x] = fetch_map[x]
        _, rec_res = self.text_classifier.postprocess(outputs, self.tmp_args)
        res = {
            "direction": [x[0] for x in rec_res],
            "score": [str(x[1]) for x in rec_res]
        }
        return res

In [ ]:
if __name__ == "__main__":
    ocr_service = OCRService(name="ocr")
    ocr_service.load_model_config(global_args.cls_model_dir)
    ocr_service.init_rec()
    if global_args.use_gpu:
        ocr_service.prepare_server(
            workdir="workdir", port=9292, device="gpu", gpuid=0)
    else:
        ocr_service.prepare_server(workdir="workdir", port=9292, device="cpu")
    ocr_service.run_rpc_service()
    ocr_service.run_web_service()